In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 경로 설정
%cd /content/drive/MyDrive/성균관대학교 과제/3학년 2학기/캡스톤설계프로젝트

/content/drive/MyDrive/24-2 Capston


In [ ]:
# 라이브러리 로드
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
import sklearn
import transformers
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import openai
from openai import OpenAI
from torch.utils.data import TensorDataset
from transformers import BertTokenizer
from tqdm import tqdm
import pickle
import math
import copy
from torch.cuda.amp import autocast, GradScaler

#import 추가
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# gpu 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##data

In [ ]:
text='안녕하세요. 저는 요즘 먹는 것 때문에 너무 힘들어서 여기에 고민을 털어놓습니다.음식을 보면 한편으로는 너무 먹고 싶다가도, 또 한편으로는 먹으면 안 된다는 생각이 머릿속에서 떠나질 않습니다. 가끔 참지 못하고 폭식하게 되면 이후에는 죄책감에 사로잡혀 아무것도 못 하겠어요. 그러다 보면 또 굶고, 몸이 버티지 못할 정도로 힘들어지는 악순환에 빠집니다. 제 몸무게나 외모에 대한 집착이 너무 커져서, 제 자신을 있는 그대로 받아들이는 게 점점 어려워지고 있어요. 사람들은 "그냥 조금만 먹으면 되잖아," "왜 그렇게 예민하게 굴어?"라고 말하지만, 이게 제 마음대로 되는 게 아니란 걸 아무도 몰라줍니다. 혹시 저처럼 이런 고민을 해보신 분들이 있다면, 어떻게 이겨낼 수 있었는지 조언 부탁드립니다. 제발 저도 자유롭게 살고 싶어요.'

###symptom identification

In [ ]:
symptoms=['음식 섭취 제한','폭식', '체중과 체형에 대한 집착', '예민함','통제력 부족']
mean_uncertainty=[0.059, 0.0541, 0.0614, 0.0569, 0.0547, 0.0572, 0.0478, 0.0625, 0.0471, 0.0478, 0.0526, 0.0498, 0.0502, 0.0595, 0.062, 0.0525, 0.0438, 0.0468, 0.0495, 0.0563, 0.0498, 0.0439, 0.0499, 0.0704, 0.0522, 0.0574, 0.0529, 0.0522, 0.0632, 0.0638, 0.0426, 0.0441, 0.0512, 0.0542, 0.0492, 0.0489, 0.0488, 0.0501, 0.0462, 0.0584, 0.0462, 0.0564, 0.0431, 0.0578, 0.0605, 0.0592, 0.0538, 0.0519]

In [ ]:
#GPT로 정의한 mean_uncertainty
mean_uncertainty=torch.tensor([0.0585, 0.049, 0.0513, 0.0496, 0.0519, 0.0501, 0.054, 0.0481, 0.0529, 0.0522, 0.0518, 0.0512, 0.0514, 0.0465, 0.0463, 0.0561,
 0.0537, 0.0495, 0.0504, 0.0608, 0.0523, 0.0522, 0.0524, 0.0529, 0.0481, 0.0509, 0.0526, 0.0488, 0.0504, 0.0519, 0.0526, 0.052,
 0.047, 0.0524, 0.049, 0.0555, 0.0493, 0.05, 0.0473, 0.0491, 0.0482, 0.0484, 0.0534, 0.0543, 0.0495, 0.0577, 0.0458, 0.0505])
print(mean_uncertainty.shape)

#차원 확장 [batch_size, num_labels]
mean_uncertainty = mean_uncertainty.unsqueeze(0)
mean_uncertainty = mean_uncertainty.expand(64, -1)
print(mean_uncertainty.shape)

torch.Size([48])
torch.Size([64, 48])


###context extraction

In [ ]:
cause=1
cause_evidence='사용자는 음식 섭취에 대한 고민과 죄책감, 외모에 대한 집착으로 인해 스트레스를 받고 있으며, 악순환에 빠져 있다고 보고함.'
frequency=0
freq_evidence='언급 없음'
duration=0
duration_evidence='언급 없음'
age=0
age_evidence='언급 없음'
social=1
social_evidence='사용자는 음식에 대한 집착과 죄책감으로 인해 사회적 관계에서 불편함을 겪고 있음'
academic=0
academic_evidence='언급 없음'
occupational=0
occupational_evidence='언급 없음'
life_threatening=0
life_threatening_evidence='언급 없음'

In [ ]:
df=pd.DataFrame([[text, symptoms, cause, frequency, duration, age, social, academic, occupational, life_threatening,
                 cause_evidence, freq_evidence, duration_evidence, age_evidence, social_evidence, academic_evidence, occupational_evidence, life_threatening_evidence]]
                , columns=['filtered_sentences','final_symptoms','cause','frequency','duration','age','social','academic','occupational','life-threatening',
                           'cause_evidence','freq_evidence','duration_evidence','age_evidence','social_evidence','academic_evidence','occupational_evidence','life-threatening_evidence'])
df

,filtered_sentences,final_symptoms,cause,frequency,duration,age,social,academic,occupational,life-threatening,cause_evidence,freq_evidence,duration_evidence,age_evidence,social_evidence,academic_evidence,occupational_evidence,life-threatening_evidence
0,안녕하세요. 저는 요즘 먹는 것 때문에 너무 힘들어서 여기에 고민을 털어놓습니다.음...,"[음식 섭취 제한, 폭식, 체중과 체형에 대한 집착, 예민함, 통제력 부족]",1,0,0,0,1,0,0,0,"사용자는 음식 섭취에 대한 고민과 죄책감, 외모에 대한 집착으로 인해 스트레스를 받...",언급 없음,언급 없음,언급 없음,사용자는 음식에 대한 집착과 죄책감으로 인해 사회적 관계에서 불편함을 겪고 있음,언급 없음,언급 없음,언급 없음


##model class

In [ ]:
# PsyEx (only risky-post stream) 모델 정의
class PsyExOnlyPostStream(nn.Module):
    def __init__(self, model_name, num_labels, num_heads=8, num_trans_layers=6, max_posts=64, freeze=False, pool_type="first") -> None:
        super().__init__()
        self.model_name = model_name
        self.num_heads = num_heads
        self.num_labels = num_labels
        self.num_trans_layers = num_trans_layers
        self.pool_type = pool_type
        self.post_encoder = AutoModel.from_pretrained(model_name)
        if freeze:
            for name, param in self.post_encoder.named_parameters():
                param.requires_grad = False
        self.hidden_dim = self.post_encoder.config.hidden_size
        self.max_posts = max_posts
        self.pos_emb = nn.Parameter(torch.Tensor(max_posts, self.hidden_dim))
        nn.init.xavier_uniform_(self.pos_emb)
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.hidden_dim, dim_feedforward=self.hidden_dim, nhead=num_heads, activation='gelu')
        self.user_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_trans_layers)
        self.attn_ff = nn.ModuleList([nn.Linear(self.hidden_dim, 1) for _ in range(self.num_labels)])
        self.dropout = nn.Dropout(self.post_encoder.config.hidden_dropout_prob)
        self.clf = nn.ModuleList([nn.Linear(self.hidden_dim, 1) for _ in range(self.num_labels)])

    def forward(self, input_ids, attention_mask):
        """
        Args:
            batch: A dictionary containing the following keys:
                - input_ids: Tensor of shape [batch_size, seq_len]
                - attention_mask: Tensor of shape [batch_size, seq_len]
                - token_type_ids: Tensor of shape [batch_size, seq_len] (optional)
        Returns:
            logits: Tensor of shape [batch_size, num_labels]
        """
        input_ids = input_ids  # [batch_size, seq_len]
        attention_mask = attention_mask  # [batch_size, seq_len]
        #token_type_ids = batch.get("token_type_ids", None)  # Optional

        # BERT Encoder
        post_outputs = self.post_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
            #token_type_ids=token_type_ids
        )

        # Pooling 처리
        if self.pool_type == "first":
            x = post_outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_dim]
        elif self.pool_type == 'mean':
            x = self.mean_pooling(post_outputs.last_hidden_state, attention_mask)  # [batch_size, hidden_dim]

        # Transformer Encoder
        x = x + self.pos_emb[:x.size(0), :]  # Add positional embeddings
        x = self.user_encoder(x.unsqueeze(1)).squeeze(1)  # [batch_size, hidden_dim]

        # Attention & Classification
        logits = []
        for i in range(self.num_labels):
            logits.append(self.clf[i](x))  # [batch_size, 1]
        logits = torch.cat(logits, dim=1)  # [batch_size, num_labels]

        return logits

    def mean_pooling(self, last_hidden_state, attention_mask):
        """
        Mean pooling using attention mask.
        Args:
            last_hidden_state: [batch_size, seq_len, hidden_dim]
            attention_mask: [batch_size, seq_len]
        Returns:
            Pooled output: [batch_size, hidden_dim]
        """
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

In [ ]:
from transformers import BertModel

class IntegratedModelWithBERT(nn.Module):
    def __init__(self, bert_model_name, embedding_dim, symptom_dim, num_labels, filter_num=64, filter_sizes=[1, 3, 5], max_pooling_k=1, dropout=0.2):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.symptom_dim = symptom_dim
        self.num_labels = num_labels

        # BERT 모델 초기화
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.bert_hidden_size = self.bert.config.hidden_size  # BERT의 출력 크기

        # CNN Layers
        self.filter_num = filter_num
        self.filter_sizes = filter_sizes
        self.hidden_size = len(filter_sizes) * filter_num
        self.convs = nn.ModuleList([nn.Conv1d(2, filter_num, size) for size in filter_sizes])  # input_channels=2

        self.max_pooling_k = max_pooling_k
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_size + self.bert_hidden_size, num_labels)  # BERT 출력과 CNN 출력 결합

    def forward(self, factor_input_ids, factor_attention_mask, symptoms):
        # BERT 출력 계산
        bert_outputs = self.bert(input_ids=factor_input_ids, attention_mask=factor_attention_mask)
        bert_pooled_output = bert_outputs.pooler_output  # (batch_size, bert_hidden_size)

        # 차원 확장: factor_input_ids에 마지막 차원 추가
        factor_input_ids = factor_input_ids.unsqueeze(-1).float()  # (batch_size, seq_len, 1)

        # Pooling으로 차원 축소: seq_len → symptom_dim
        factors_reduced = F.adaptive_avg_pool1d(factor_input_ids.transpose(1, 2), output_size=self.symptom_dim).transpose(1, 2)  # (batch_size, symptom_dim, 1)

        # 증상 데이터 차원 맞춤
        input_seqs = torch.cat([symptoms, factors_reduced], dim=2)  # (batch_size, symptom_dim, 2)

        # Conv1d 입력 차원 변환
        input_seqs = input_seqs.transpose(1, 2)  # (batch_size, 2, symptom_dim)

        # CNN 처리
        x = [F.relu(conv(input_seqs)) for conv in self.convs]
        x = [self.kmax_pooling(item, self.max_pooling_k).mean(2) for item in x]
        x = torch.cat(x, 1)  # (batch_size, hidden_size)
        x = self.dropout(x)

        # BERT 출력과 CNN 출력을 결합
        combined = torch.cat([bert_pooled_output, x], dim=1)  # (batch_size, hidden_size + bert_hidden_size)

        # Fully Connected Layer for prediction
        logits = self.fc(combined)  # (batch_size, num_labels)
        return logits

    def kmax_pooling(self, x, k):
        if x.dim() < 3:
            raise ValueError(f"Input to kmax_pooling must have at least 3 dimensions, but got {x.shape}")
        return x.sort(dim=2, descending=True)[0][:, :, :k]


In [ ]:
# 모델 정의
class Symp(nn.Module):
    def __init__(self, input_dim, filter_num=128, filter_sizes=[1], dropout=0.2, num_labels=16, k_max_pooling=1):
        super(Symp, self).__init__()
        self.filter_num = filter_num
        self.filter_sizes = filter_sizes
        self.hidden_size = len(filter_sizes) * filter_num
        self.convs = nn.ModuleList([nn.Conv1d(input_dim, filter_num, size) for size in filter_sizes])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_size, num_labels)
        self.k_max_pooling = k_max_pooling

    def forward(self, input_seqs):
        # Conv1d 적용 및 활성화 함수
        x = [F.relu(conv(input_seqs)) for conv in self.convs]

        # k-max pooling 적용
        x = [self.kmax_pooling(item, self.k_max_pooling) for item in x]

        # 평균값 계산 및 결합
        x = [item.mean(dim=2) for item in x]  # [batch_size, filter_num]
        x = torch.cat(x, 1)  # [batch_size, len(filter_sizes) * filter_num]

        # 드롭아웃 및 최종 출력
        x = self.dropout(x)
        logits = self.fc(x)  # [batch_size, num_labels]
        return torch.sigmoid(logits)  # Sigmoid 활성화 함수 적용


    def kmax_pooling(self, x, k):
        return x.sort(dim=2, descending=True)[0][:, :, :k]

In [ ]:
# 모델 정의
class SympWithFactor(nn.Module):
    def __init__(self, input_dim, num_factors, filter_num=64, filter_sizes=[1], dropout=0.2, num_labels=16, max_pooling_k=1):
        super(SympWithFactor, self).__init__()
        self.filter_num = filter_num
        self.filter_sizes = filter_sizes
        self.hidden_size = len(filter_sizes) * filter_num
        self.max_pooling_k = max_pooling_k
        self.convs = nn.ModuleList([nn.Conv1d(16, filter_num, size) for size in filter_sizes]) #num_factors * 2->16
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_size, num_labels)

    def forward(self, symptoms, factors):
        # symptoms를 축소하여 factors와 동일한 크기로 조정
        symptoms_reduced = self.kmax_pooling(symptoms.unsqueeze(1), k=factors.size(1)).squeeze(1)  # [batch_size, num_factors]

        # 축소된 symptoms와 factors를 결합
        input_seqs = torch.cat([symptoms_reduced, factors], dim=1).unsqueeze(1)  # [batch_size, 1, num_factors * 2]

        # Conv1d에 맞게 차원 전환
        input_seqs = input_seqs.transpose(1, 2)  # [batch_size, num_factors * 2, 1]
        x = [F.relu(conv(input_seqs)) for conv in self.convs]
        x = [self.kmax_pooling(item, self.max_pooling_k).mean(2) for item in x]
        x = torch.cat(x, 1)
        x = self.dropout(x)
        logits = self.fc(x)
        return logits

    def kmax_pooling(self, x, k):
        return x.sort(dim=2, descending=True)[0][:, :, :k]

In [ ]:
class CURE(nn.Module):
    def __init__(self, target_models, target_uncertainty, predictors, hidden_dim, num_labels, num_models, dropout_ratio, freeze=True):
        super(CURE, self).__init__()
        # Hyper-parameters
        self.num_labels = num_labels
        self.num_uncertainty = len(target_uncertainty)
        self.hidden_dim = hidden_dim
        self.num_models = len(target_models)
        # Model predictions:: sub-models
        self.target_models_name = target_models
        self.target_uncertainty = target_uncertainty
        self.predictors = predictors
        # Uncertainty-aware decision fusion layers
        self.linear1 = nn.Linear(112, self.hidden_dim)
        #self.linear1 = nn.Linear(self.num_models * self.num_labels + self.num_uncertainty, self.hidden_dim)
        self.linear2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.logits = nn.Linear(self.hidden_dim, self.num_labels)
        self.dropout = nn.Dropout(dropout_ratio)
        # Optional: Freeze sub-models
        self.reset_parameters()
        if freeze:
            for model in self.predictors:
                for name, param in model.named_parameters():
                    param.requires_grad = False

    def reset_parameters(self): # Initialize weights
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)
        nn.init.xavier_uniform_(self.logits.weight)

    def make_prediction(self, model_type, predictor, input_data):
        if model_type in ["bertq"]:
            input_ids, attention_mask = input_data
            logits = predictor(input_ids, attention_mask)
            #print("bertq OK")
            return torch.sigmoid(logits)
        if model_type in ["bertc"]:
            input_ids, attention_mask, symptom = input_data
            logits = predictor(input_ids, attention_mask, symptom)
            #print("bertc OK")
            return torch.sigmoid(logits)
        if model_type in ["sympc"]:
            symptom, factor = input_data
            logits = predictor(symptom, factor)
            #print("sympc OK")
            return torch.sigmoid(logits)
        else:  # "symp"
            logits = predictor(input_data)
            #print("sympp OK")
            return torch.sigmoid(logits)

    def forward(self, text_input_ids, text_attention_mask, factor_input_ids,
                factor_attention_mask, symptom, factors, symptom_uncertainty):

        '''print(f"text_input_ids: {text_input_ids.shape}")
        print(f"text_attention_mask: {text_attention_mask.shape}")
        print(f"factor_input_ids: {factor_input_ids.shape}")
        print(f"factor_attention_mask: {factor_attention_mask.shape}")
        print(f"symptom: {symptom.shape}")
        print(f"factors: {factors.shape}")
        print(f"symptom_uncertainty: {symptom_uncertainty.shape}")'''

        # Setting variables
        #swfactor = torch.cat([symptom, factors], dim=1).unsqueeze(2)
        #symptom = symptom.unsqueeze(2).squeeze(0)
        uncertainties = []
        pred_logits = []

        # Model predictions
        with torch.no_grad():
            for model_type, predictor in zip(self.target_models_name, self.predictors):
                if model_type == "bertq":
                    input_data = (text_input_ids, text_attention_mask)
                elif model_type == "bertc":
                    input_data = (factor_input_ids, factor_attention_mask, symptom.unsqueeze(2).squeeze(0))
                elif model_type == "symp":
                    input_data = symptom.unsqueeze(2).squeeze(0)
                elif model_type == "sympc":
                    input_data = (symptom, factors)  # 기존 코드랑 다름??
                else:
                    raise ValueError("Invalid Model Type")

                pred_logit = self.make_prediction(model_type, predictor, input_data)
                pred_logits.append(pred_logit)

        # Uncertainty-aware decision fusion
        # symptom_uncertainty는 외부에서 받은 불확실성
        uncertainties.append(symptom_uncertainty.unsqueeze(1))

        '''# BertMultiLabelClassificationWithSNGP에서 얻은 불확실성
        for predictor in self.predictors:
            if isinstance(predictor, BertMultiLabelClassificationWithSNGP):
                # SNGP 모델에서 추출한 불확실성 값 받기
                mean_uncertainty = predictor.uncertainty  # 모델에서 추출한 불확실성 값
                uncertainties.append(mean_uncertainty.unsqueeze(1))'''

        uncertainties = torch.cat(uncertainties, dim=1)  # [batch size, num_uncertainty]

        stacked_all = torch.cat(pred_logits, dim=1)  # [batch_size, num_labels * num_models]
        cat = torch.cat([uncertainties.squeeze(1), stacked_all], dim=1)  # [batch_size, (num_labels * num_models) + num_uncertainty]

        hidden = self.linear1(cat)
        hidden = F.elu(self.dropout(hidden))
        hidden = self.linear2(hidden)
        hidden = F.elu(self.dropout(hidden))
        logits = self.logits(hidden)

        return logits

In [ ]:
import torch

# CUDA 장치에서 로드
with open('1205_model/bert_post.pkl', 'rb') as f:
    BERT_post = torch.load(f)

with open('최종모델/bert_context_최종.pkl', 'rb') as f:
    BERT_context = torch.load(f)

with open('최종모델/symp_symp_최종.pkl', 'rb') as f:
    SYMP_symp = torch.load(f)

with open('최종모델/symp_context_최종.pkl', 'rb') as f:
    SYMP_context = torch.load(f)

with open('최종모델/cure_최종.pkl', 'rb') as f:
    cure = torch.load(f)

<ipython-input-15-859cea5787a6>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  BERT_post = torch.load(f)
<ipython-input-15-859cea5787a6>:8: FutureWarning: You are using `t

##CURE

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, mean_uncertainty, max_length=128, batch_size=64):
        self.tokenizer = tokenizer
        self.mean_uncertainty = mean_uncertainty
        self.max_length = max_length
        self.batch_size = batch_size

        # 복제 로직: 데이터를 batch_size에 맞게 복제
        self.df = pd.concat([df] * ((batch_size + len(df) - 1) // len(df)), ignore_index=True)
        self.mean_uncertainty = mean_uncertainty * ((batch_size + len(mean_uncertainty) - 1) // len(mean_uncertainty))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # 텍스트 필드 가져오기 및 문자열 변환
        filtered_sentences = str(row['filtered_sentences'])
        final_symptoms = str(row['final_symptoms'])

        # Evidence 필드 처리
        evidence_fields = [
            str(evidence) if not pd.isna(evidence) else ""  # NaN을 빈 문자열로 대체
            for evidence in [
                row['cause_evidence'], row['freq_evidence'], row['duration_evidence'],
                row['age_evidence'], row['social_evidence'], row['academic_evidence'],
                row['occupational_evidence'], row['life-threatening_evidence']
            ]
        ]

        # Evidence 토크나이징
        evidence_input_ids_list = [
            self.tokenizer.encode(evidence, add_special_tokens=True, truncation=True,
                                  max_length=self.max_length, padding='max_length')
            for evidence in evidence_fields
        ]

        # Attention Mask 생성
        def create_attention_mask(input_ids):
            return [1 if token_id != 0 else 0 for token_id in input_ids]

        evidence_attention_mask_list = [create_attention_mask(ids) for ids in evidence_input_ids_list]

        # Factor 증거 결합
        factor_input_ids, factor_attention_mask = [], []
        for input_ids, attn_mask in zip(evidence_input_ids_list, evidence_attention_mask_list):
            factor_input_ids.extend(input_ids)
            factor_attention_mask.extend(attn_mask)

        factor_input_ids = factor_input_ids[:self.max_length]
        factor_attention_mask = factor_attention_mask[:self.max_length]

        # 텍스트 토크나이징
        text_inputs = self.tokenizer(
            filtered_sentences,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # 질병 레이블 텐서 생성
        symptom_list = ['무증상', '짜증이 잘 남', '두려움과 공포', '호흡곤란', '폭식', '답답함', '초조하고 안절부절 못함', '언어 능력 감소',
                        '피해망상', '불면', '성에 대한 과도한 관심', '우울함', '강박사고', '음식 섭취 제한', '자살사고', '다리 불편감', '기억력 저하',
                        '체중과 체형에 대한 집착', '무기력', '주의산만', '긴장', '감정 기복', '환각', '수면 무호흡증', '환청', '수면 과다', '충동성',
                        '피로', '메스꺼움', '어지러움', '눈물 흘림', '예민함', '극단적인 생각', '반복행동', '구토와 약물', '수면문제', '불안함',
                        '자극적인 행위에 몰입', '통제력 부족', '분노 조절 문제', '발작', '두통', '가슴 두근거림', '장시간 지속되는 이유 없는 불안감',
                        '집중력 저하', '일상생활 능력의 손상', '식욕감소', '걱정이 많아짐']
        symptom_labels = final_symptoms
        symptom_name_to_index = {name: idx for idx, name in enumerate(symptom_list)}

        symptom_tensor = torch.zeros(len(symptom_list))
        for label in symptom_labels:
            if label in symptom_name_to_index:
                symptom_tensor[symptom_name_to_index[label]] = 1

        # Factors 텐서 생성
        factors = {
            'cause': row['cause'],
            'frequency': row['frequency'],
            'duration': row['duration'],
            'age': row['age'],
            'social': row['social'],
            'academic': row['academic'],
            'occupational': row['occupational'],
            'life-threatening': row['life-threatening']
        }
        factors_tensor = torch.tensor([factors[key] for key in factors], dtype=torch.float32)

        # Uncertainty 값 가져오기
        uncertainty = self.mean_uncertainty[min(idx, len(self.mean_uncertainty) - 1)]

        return {
            'sentences_input_ids': text_inputs['input_ids'].squeeze(0),
            'sentences_attention_mask': text_inputs['attention_mask'].squeeze(0),
            'final_symptoms': symptom_tensor,
            'factor_input_ids': torch.tensor(factor_input_ids, dtype=torch.long),
            'factor_attention_mask': torch.tensor(factor_attention_mask, dtype=torch.long),
            'factors': factors_tensor,
            'symptom_uncertainty': torch.tensor(uncertainty, dtype=torch.float)
        }


In [ ]:
from torch.utils.data import DataLoader

tokenizer=AutoTokenizer.from_pretrained('klue/bert-base')

# CustomDataset 객체 생성
dataset = CustomDataset(df, tokenizer, mean_uncertainty)

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

for batch in dataloader:
    print(f"sentences_input_ids shape: {batch['sentences_input_ids'].shape}")
    print(f"sentences_attention_mask shape: {batch['sentences_attention_mask'].shape}")
    print(f"final_symptoms shape: {batch['final_symptoms'].shape}")
    print(f"factor_input_ids shape: {batch['factor_input_ids'].shape}")
    print(f"factor_attention_mask shape: {batch['factor_attention_mask'].shape}")
    print(f"factors shape: {batch['factors'].shape}")
    print(f"symptom_uncertainty shape: {batch['symptom_uncertainty'].shape}")
    break

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-18-b956d9827d72>:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'symptom_uncertainty': torch.tensor(uncertainty, dtype=torch.float)


sentences_input_ids shape: torch.Size([64, 128])
sentences_attention_mask shape: torch.Size([64, 128])
final_symptoms shape: torch.Size([64, 48])
factor_input_ids shape: torch.Size([64, 128])
factor_attention_mask shape: torch.Size([64, 128])
factors shape: torch.Size([64, 8])
symptom_uncertainty shape: torch.Size([64, 48])


In [ ]:
# 모델 검증
average_threshold = 0.4  # 모델 출력의 평균 확률값을 기준으로 설정

predictions, true_labels = [], []  # 예측값과 실제값 저장 리스트

with torch.no_grad():  # 그래디언트 계산 비활성화
    for batch in dataloader:#train_loader:
        # 텍스트 (입력 시퀀스)
        sentences_input_ids = batch["sentences_input_ids"].to(device)
        sentences_attention_mask = batch["sentences_attention_mask"].to(device)
        factor_input_ids = batch["factor_input_ids"].to(device)
        factor_attention_mask = batch["factor_attention_mask"].to(device)
        symptom = batch["final_symptoms"].to(device)  # 증상(symptom) 텍스트
        factors = batch["factors"].to(device)  # factor는 여러 증거 텍스트를 결합한 것
        symptom_uncertainty = batch["symptom_uncertainty"].to(device)

        # 모델 예측
        logits = cure(
            text_input_ids=sentences_input_ids,
            text_attention_mask=sentences_attention_mask,
            factor_input_ids=factor_input_ids,
            factor_attention_mask=factor_attention_mask,
            symptom=symptom,
            factors=factors,
            symptom_uncertainty=symptom_uncertainty
        )
        probabilities = torch.sigmoid(logits)  # 로짓 값을 확률로 변환

        # 임계값을 기준으로 이진화
        predicted_labels = (probabilities > average_threshold).int()
        predictions.append(predicted_labels)

print(predictions[0])

<ipython-input-18-b956d9827d72>:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'symptom_uncertainty': torch.tensor(uncertainty, dtype=torch.float)


tensor([[0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int32)


In [ ]:
predictions[0][0]

#임계값 평균으로 했을 때: 수면장애 섭식장애 스트레스 불안장애 ADHD 공황장애 틱장애 조현병
#임계값 더 올려서 0.4로 했을 때: 섭식장애

tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0',
       dtype=torch.int32)

In [ ]:
#["수면장애", "우울증", "섭식장애", "스트레스", "불안장애", "ADHD", "Non-disease", "공황장애",
                   "강박장애", "사회 불안장애", "중독(의존)", "하지 불안 증후군", "치매", "틱장애", "조현병", "양극성장애"]